# 2.0 ANTsPy 配准测试

本 notebook 用于测试 ANTsPy 的配准功能。

## 目标
1. 验证 ANTsPy 安装
2. 测试基本配准功能
3. 可视化配准结果

In [ ]:
import sys
sys.path.append('..')

import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

# 检查 ANTsPy
try:
    import ants
    print(f"ANTsPy 版本: {ants.__version__}")
except ImportError:
    print("ANTsPy 未安装，请运行: pip install antspyx")

## 1. 加载测试数据

In [ ]:
# 配置路径
DATA_DIR = Path('../data/01_cleaned')
ATLAS_DIR = Path('../data/02_atlas')

# 查找可用的 CT 文件
ct_files = list(DATA_DIR.rglob('*.nii.gz'))
print(f"找到 {len(ct_files)} 个 CT 文件")

In [ ]:
# 加载两个 CT 进行配准测试
if len(ct_files) >= 2:
    fixed = ants.image_read(str(ct_files[0]))
    moving = ants.image_read(str(ct_files[1]))
    
    print(f"Fixed: {fixed.shape}, spacing={fixed.spacing}")
    print(f"Moving: {moving.shape}, spacing={moving.spacing}")
else:
    print("需要至少 2 个 CT 文件进行配准测试")

## 2. 刚性配准测试

In [ ]:
if len(ct_files) >= 2:
    print("执行刚性配准...")
    rigid_result = ants.registration(
        fixed=fixed,
        moving=moving,
        type_of_transform='Rigid',
        verbose=True
    )
    print("刚性配准完成!")

## 3. SyN 非刚性配准测试

In [ ]:
if len(ct_files) >= 2:
    print("执行 SyN 配准（这可能需要几分钟）...")
    syn_result = ants.registration(
        fixed=fixed,
        moving=moving,
        type_of_transform='SyNRA',
        reg_iterations=[40, 20, 10],
        verbose=True
    )
    print("SyN 配准完成!")

## 4. 可视化配准结果

In [ ]:
def visualize_registration(fixed, moving, warped, slice_idx=None):
    """可视化配准结果"""
    if slice_idx is None:
        slice_idx = fixed.shape[0] // 2
    
    fig, axes = plt.subplots(1, 4, figsize=(16, 4))
    
    axes[0].imshow(fixed.numpy()[slice_idx], cmap='gray')
    axes[0].set_title('Fixed')
    
    axes[1].imshow(moving.numpy()[slice_idx], cmap='gray')
    axes[1].set_title('Moving')
    
    axes[2].imshow(warped.numpy()[slice_idx], cmap='gray')
    axes[2].set_title('Warped')
    
    # 差异图
    diff = np.abs(fixed.numpy()[slice_idx] - warped.numpy()[slice_idx])
    axes[3].imshow(diff, cmap='hot')
    axes[3].set_title('Difference')
    
    for ax in axes:
        ax.axis('off')
    
    plt.tight_layout()
    plt.show()

In [ ]:
if len(ct_files) >= 2:
    visualize_registration(fixed, moving, syn_result['warpedmovout'])

## 5. 计算配准质量指标

In [ ]:
if len(ct_files) >= 2:
    # 计算相关性
    corr_before = ants.image_similarity(fixed, moving, metric_type='Correlation')
    corr_after = ants.image_similarity(fixed, syn_result['warpedmovout'], metric_type='Correlation')
    
    print(f"配准前相关性: {corr_before:.4f}")
    print(f"配准后相关性: {corr_after:.4f}")
    print(f"改善: {(corr_after - corr_before) / abs(corr_before) * 100:.1f}%")